In [ ]:
import numpy as np
import pandas as pd
import re
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Conv1D, MaxPooling1D, Dense
from keras.models import Sequential
from sklearn.metrics import classification_report

# 載入與預處理數據
def load_semeval_data(file_paths):
    data = []
    labels = []
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split('\t')
                if len(parts) == 3:
                    data.append(parts[2])
                    labels.append(parts[1])
    return pd.DataFrame({'tweet': data, 'label': labels})

file_paths = [
    './dataset/train/twitter-2013train-A.txt',
    './dataset/train/twitter-2015train-A.txt',
    './dataset/train/twitter-2016train-A.txt',
]

dataset = load_semeval_data(file_paths)

# 預處理推文文本
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"https?:\/\/\S+", "", tweet)
    tweet = re.sub(r"[^a-zA-Z0-9\s]", "", tweet)
    return tweet

dataset['tweet'] = dataset['tweet'].apply(preprocess_tweet)

# Tokenization 和 Padding 設置
MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 40  # 用於 N-R 模式
REGION_SIZE = 10  # 每個區域的長度，用於 R 模式
NUM_REGIONS = 4  # 區域數量，用於 R 模式
EMBEDDING_DIM = 25

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(dataset['tweet'].values)
word_index = tokenizer.word_index

mode = 'R'  # 可以選擇 'N-R' 或 'R'

# 根據模式選擇不同的數據處理方法
if mode == 'N-R':
    # 推文轉為固定長度序列
    X = tokenizer.texts_to_sequences(dataset['tweet'].values)
    X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
    input_length = MAX_SEQUENCE_LENGTH  # 設置模型的 input_length
elif mode == 'R':
    # 推文分成多個區域並填充至相同長度
    X = tokenizer.texts_to_sequences(dataset['tweet'].values)
    X_padded = []
    for tweet in X:
        tweet_regions = []
        for i in range(NUM_REGIONS):
            start = i * REGION_SIZE
            end = start + REGION_SIZE
            region = tweet[start:end]
            if len(region) < REGION_SIZE:
                region = region + [0] * (REGION_SIZE - len(region))
            tweet_regions.extend(region)
        X_padded.append(tweet_regions)
    X = np.array(X_padded)
    input_length = NUM_REGIONS * REGION_SIZE  # 設置模型的 input_length
else:
    raise ValueError("Invalid mode. Choose 'N-R' or 'R'.")

# 標籤處理
label_mapping = {'positive': 0, 'negative': 1, 'neutral': 2}
y = dataset['label'].map(label_mapping).values
y = pd.get_dummies(y).values

# 數據分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 訓練 Word2Vec 模型
tokenized_sentences = [tweet.split() for tweet in dataset['tweet'].values]
word2vec_model = Word2Vec(sentences=tokenized_sentences,
                          vector_size=EMBEDDING_DIM,  # 詞向量維度為 25
                          window=5,                   # 最大跳躍距離設為 5
                          min_count=5,                # 最低頻次為 5
                          sg=0,                       # CBOW 模型 (sg=0)
                          workers=4)                  # 使用 4 個核心進行訓練

# 將詞向量進行標準化
def normalize_vector(vector):
    v_min, v_max = np.min(vector), np.max(vector)
    return (vector - v_min) / (v_max - v_min) if v_max > v_min else vector

# 構建嵌入矩陣
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_vector = word2vec_model.wv[word]
        embedding_matrix[i] = normalize_vector(embedding_vector)

# 定義嵌入層，用於後續模型
embedding_layer = Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                            input_length=input_length, trainable=False)

### 建立單一 CNN 模型並加入 Flatten 層

In [2]:
from keras.layers import Flatten

cnn_model = Sequential()
cnn_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                        input_length=MAX_SEQUENCE_LENGTH, trainable=False))
cnn_model.add(Conv1D(filters=12, kernel_size=3, activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=3))
cnn_model.add(Flatten())  # 加入 Flatten 層
cnn_model.add(Dense(3, activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 2s 3ms/step - loss: 1.0154 - accuracy: 0.4439 - val_loss: 0.9926 - val_accuracy: 0.4742
Epoch 2/10
405/405 [==============================] - 1s 2ms/step - loss: 0.9984 - accuracy: 0.4660 - val_loss: 0.9855 - val_accuracy: 0.4801
Epoch 3/10
405/405 [==============================] - 1s 2ms/step - loss: 0.9912 - accuracy: 0.4683 - val_loss: 0.9813 - val_accuracy: 0.4785
Epoch 4/10
405/405 [==============================] - 1s 2ms/step - loss: 0.9854 - accuracy: 0.4771 - val_loss: 0.9791 - val_accuracy: 0.4813
Epoch 5/10
405/405 [==============================] - 1s 2ms/step - loss: 0.9797 - accuracy: 0.4849 - val_loss: 0.9735 - val_accuracy: 0.4859
Epoch 6/10
405/405 [==============================] - 1s 2ms/step - loss: 0.9761 - accuracy: 0.4915 - val_loss: 0.9697 - val_accuracy: 0.4819
Epoch 7/10
405/405 [==============================] - 1s 2ms/step - loss: 0.9726 - accuracy: 0.4954 - val_loss: 0.9663 - val_accuracy: 0.4909
Epoch 

### 單一 LSTM 模型 (Single LSTM)

In [ ]:
from keras.layers import LSTM

lstm_model = Sequential()
lstm_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                         input_length=MAX_SEQUENCE_LENGTH, trainable=False))
lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(3, activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 24s 55ms/step - loss: 1.0091 - accuracy: 0.4364 - val_loss: 0.9856 - val_accuracy: 0.4634
Epoch 2/10
405/405 [==============================] - 24s 59ms/step - loss: 0.9972 - accuracy: 0.4490 - val_loss: 0.9844 - val_accuracy: 0.4615
Epoch 3/10
405/405 [==============================] - 27s 66ms/step - loss: 0.9935 - accuracy: 0.4658 - val_loss: 0.9850 - val_accuracy: 0.4563
Epoch 4/10
405/405 [==============================] - 27s 67ms/step - loss: 0.9908 - accuracy: 0.4603 - val_loss: 0.9811 - val_accuracy: 0.4624
Epoch 5/10
405/405 [==============================] - 30s 73ms/step - loss: 0.9903 - accuracy: 0.4611 - val_loss: 0.9769 - val_accuracy: 0.4615
Epoch 6/10
405/405 [==============================] - 29s 72ms/step - loss: 0.9911 - accuracy: 0.4617 - val_loss: 0.9718 - val_accuracy: 0.4652
Epoch 7/10
405/405 [==============================] - 61s 151ms/step - loss: 0.9914 - accuracy: 0.4620 - val_loss: 0.9815 - val_accuracy

c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### CNN + LSTM 組合模型 (CNN + LSTM Combined Network)

In [ ]:
cnn_lstm_model = Sequential()
cnn_lstm_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                             input_length=MAX_SEQUENCE_LENGTH, trainable=False))
cnn_lstm_model.add(Conv1D(filters=12, kernel_size=3, activation='relu'))
cnn_lstm_model.add(MaxPooling1D(pool_size=3))
cnn_lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
cnn_lstm_model.add(Dense(3, activation='softmax'))
cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 14s 26ms/step - loss: 1.0069 - accuracy: 0.4424 - val_loss: 0.9872 - val_accuracy: 0.4733
Epoch 2/10
405/405 [==============================] - 10s 24ms/step - loss: 0.9881 - accuracy: 0.4743 - val_loss: 0.9760 - val_accuracy: 0.4992
Epoch 3/10
405/405 [==============================] - 10s 24ms/step - loss: 0.9815 - accuracy: 0.4846 - val_loss: 0.9633 - val_accuracy: 0.4825
Epoch 4/10
405/405 [==============================] - 10s 24ms/step - loss: 0.9778 - accuracy: 0.4883 - val_loss: 0.9615 - val_accuracy: 0.4832
Epoch 5/10
405/405 [==============================] - 10s 25ms/step - loss: 0.9759 - accuracy: 0.4862 - val_loss: 0.9582 - val_accuracy: 0.4955
Epoch 6/10
405/405 [==============================] - 10s 25ms/step - loss: 0.9742 - accuracy: 0.4869 - val_loss: 0.9753 - val_accuracy: 0.4757
Epoch 7/10
405/405 [==============================] - 10s 25ms/step - loss: 0.9707 - accuracy: 0.4950 - val_loss: 0.9528 - val_accuracy:

c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### CNN + LSTM 組合模型 + SVM 組合模型

In [5]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# CNN 特徵提取模型
cnn_feature_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=12, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=3),
    Flatten()
])

# LSTM 特徵提取模型
lstm_feature_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Flatten()
])

# 提取 CNN 和 LSTM 特徵
cnn_features = cnn_feature_model.predict(X_train)
lstm_features = lstm_feature_model.predict(X_train)
combined_features = np.hstack((cnn_features, lstm_features))

# 使用 SVM 進行分類
svm_classifier = SVC(kernel='linear', probability=True)
svm_classifier.fit(combined_features, np.argmax(y_train, axis=1))

# 評估
cnn_test_features = cnn_feature_model.predict(X_test)
lstm_test_features = lstm_feature_model.predict(X_test)
combined_test_features = np.hstack((cnn_test_features, lstm_test_features))
y_pred = svm_classifier.predict(combined_test_features)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred))

102/102 [==============================] - 7s 67ms/step
              precision    recall  f1-score   support

           0       0.48      0.57      0.52      1372
           1       1.00      0.00      0.00       466
           2       0.50      0.58      0.53      1397

    accuracy                           0.49      3235
   macro avg       0.66      0.38      0.35      3235
weighted avg       0.56      0.49      0.45      3235



### 單一 3 層 CNN 和 LSTM 網絡

In [6]:
three_layer_cnn_lstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

three_layer_cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
three_layer_cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = three_layer_cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 197s 476ms/step - loss: 1.0035 - accuracy: 0.4470 - val_loss: 0.9861 - val_accuracy: 0.4532
Epoch 2/10
405/405 [==============================] - 192s 474ms/step - loss: 0.9907 - accuracy: 0.4644 - val_loss: 0.9826 - val_accuracy: 0.4594
Epoch 3/10
405/405 [==============================] - 267s 661ms/step - loss: 0.9848 - accuracy: 0.4780 - val_loss: 0.9853 - val_accuracy: 0.4692
Epoch 4/10
405/405 [==============================] - 8s 19ms/step - loss: 0.9782 - accuracy: 0.4869 - val_loss: 0.9802 - val_accuracy: 0.4714
Epoch 5/10
405/405 [==============================] - 8s 19ms/step - loss: 0.9702 - accuracy: 0.4878 - val_loss: 0.9683 - val_accuracy: 0.4862
Epoch 6/10
405/405 [==============================] - 7s 17ms/step - loss: 0.9631 - accuracy: 0.4982 - val_loss: 0.9752 - val_accuracy: 0.4881
Epoch 7/10
405/405 [==============================] - 7s 16ms/step - loss: 0.9599 - accuracy: 0.5018 - val_loss: 0.9730 - val_accurac

### 多重 CNN 和 LSTM 網絡

In [7]:
multi_cnn_lstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

multi_cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
multi_cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = multi_cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 14s 25ms/step - loss: 1.0090 - accuracy: 0.4412 - val_loss: 1.0013 - val_accuracy: 0.4541
Epoch 2/10
405/405 [==============================] - 10s 25ms/step - loss: 0.9903 - accuracy: 0.4687 - val_loss: 0.9730 - val_accuracy: 0.4733
Epoch 3/10
405/405 [==============================] - 13s 33ms/step - loss: 0.9805 - accuracy: 0.4845 - val_loss: 0.9730 - val_accuracy: 0.4896
Epoch 4/10
405/405 [==============================] - 146s 359ms/step - loss: 0.9762 - accuracy: 0.4857 - val_loss: 0.9683 - val_accuracy: 0.4900
Epoch 5/10
405/405 [==============================] - 11s 28ms/step - loss: 0.9718 - accuracy: 0.4903 - val_loss: 0.9921 - val_accuracy: 0.4606
Epoch 6/10
405/405 [==============================] - 12s 29ms/step - loss: 0.9694 - accuracy: 0.4984 - val_loss: 0.9570 - val_accuracy: 0.4974
Epoch 7/10
405/405 [==============================] - 12s 31ms/step - loss: 0.9668 - accuracy: 0.4989 - val_loss: 0.9576 - val_accurac

### 單一 3 層 CNN 和雙向 LSTM 網絡

In [8]:
from keras.layers import Bidirectional

three_layer_cnn_bilstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(3, activation='softmax')
])

three_layer_cnn_bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
three_layer_cnn_bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = three_layer_cnn_bilstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 17s 23ms/step - loss: 1.0051 - accuracy: 0.4491 - val_loss: 0.9779 - val_accuracy: 0.4683
Epoch 2/10
405/405 [==============================] - 9s 21ms/step - loss: 0.9871 - accuracy: 0.4727 - val_loss: 0.9812 - val_accuracy: 0.4770
Epoch 3/10
405/405 [==============================] - 9s 23ms/step - loss: 0.9787 - accuracy: 0.4862 - val_loss: 0.9702 - val_accuracy: 0.4754
Epoch 4/10
405/405 [==============================] - 9s 23ms/step - loss: 0.9725 - accuracy: 0.4913 - val_loss: 0.9814 - val_accuracy: 0.4881
Epoch 5/10
405/405 [==============================] - 9s 23ms/step - loss: 0.9660 - accuracy: 0.4967 - val_loss: 0.9681 - val_accuracy: 0.4887
Epoch 6/10
405/405 [==============================] - 10s 24ms/step - loss: 0.9616 - accuracy: 0.5038 - val_loss: 0.9602 - val_accuracy: 0.4961
Epoch 7/10
405/405 [==============================] - 9s 23ms/step - loss: 0.9514 - accuracy: 0.5130 - val_loss: 0.9675 - val_accuracy: 0.48

### 多重 CNN 和雙向 LSTM 網絡

In [9]:
multi_cnn_bilstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(3, activation='softmax')
])

multi_cnn_bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
multi_cnn_bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = multi_cnn_bilstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 24s 41ms/step - loss: 1.0029 - accuracy: 0.4563 - val_loss: 0.9748 - val_accuracy: 0.4943
Epoch 2/10
405/405 [==============================] - 17s 43ms/step - loss: 0.9892 - accuracy: 0.4692 - val_loss: 0.9773 - val_accuracy: 0.4569
Epoch 3/10
405/405 [==============================] - 18s 44ms/step - loss: 0.9754 - accuracy: 0.4866 - val_loss: 0.9679 - val_accuracy: 0.4606
Epoch 4/10
405/405 [==============================] - 18s 45ms/step - loss: 0.9698 - accuracy: 0.4926 - val_loss: 0.9730 - val_accuracy: 0.4631
Epoch 5/10
405/405 [==============================] - 18s 46ms/step - loss: 0.9671 - accuracy: 0.4962 - val_loss: 0.9589 - val_accuracy: 0.4952
Epoch 6/10
405/405 [==============================] - 19s 48ms/step - loss: 0.9642 - accuracy: 0.4995 - val_loss: 0.9650 - val_accuracy: 0.4733
Epoch 7/10
405/405 [==============================] - 19s 48ms/step - loss: 0.9587 - accuracy: 0.5036 - val_loss: 0.9590 - val_accuracy: